In [1]:
import os
from decouple import config
from langchain_groq import ChatGroq
import os


In [2]:
os.environ['GROQ_API_KEY']=config("GROQ_API_KEY")


In [3]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI

loader = WebBaseLoader("https://en.wikipedia.org/wiki/Parul_University")
docs = loader.load()
docs


USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Parul_University', 'title': 'Parul University - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nParul University - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nHistory, gove

In [4]:
from langchain_text_splitters import TokenTextSplitter
from langchain_core.prompts import PromptTemplate

text_splitter = TokenTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=10
)
split_docs = text_splitter.split_documents(docs)
print(f"Number of chunk created after splitting :- {len(split_docs)}")


Number of chunk created after splitting :- 9


In [5]:
# chat = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")
# system = "Generate summary in 90 words"
# human = "{text}"
# prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

# chain = prompt | chat
# chain.invoke({"text": f"{docs}"})


In [6]:
def summarize_chunks(split_docs, model_name="gemma2-9b-it"):
    # Initialize the Groq LLM
    llm = ChatGroq(model_name=model_name)

    # Create a prompt template for individual chunk summarization
    individual_prompt = PromptTemplate(
        input_variables=["text"],
        template="Summarize the following text:\n\n{text}\n\nSummary:"
    )

    # Create a prompt template for the final summary
    final_prompt = PromptTemplate(
        input_variables=["text"],
        template="Combine the following summaries into a coherent overall summary:\n\n{text}\n\n directly give the summary without any extra prefix token"
    )

    # Create the summarization chain for individual chunks
    chain = load_summarize_chain(
        llm,
        chain_type="map_reduce",
        map_prompt=individual_prompt,
        combine_prompt=final_prompt,
        verbose=True
    )

    # Convert chunks to Document objects
    # docs = [Document(page_content=chunk) for chunk in chunks]

    # Generate the final summary
    final_summary = chain.invoke(split_docs)

    return final_summary


In [7]:
summary = summarize_chunks(split_docs)
summary




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Summarize the following text:





Parul University - Wikipedia



























Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload file



















Search











Search






















Appearance
















Create account

Log in








Personal tools





 Create account Log in





		Pages for logged out editors learn more



ContributionsTalk




























Contents
move to sidebar
hide




(Top)





1
History, governance and leadership








2
Affiliations and accreditations








3
Parul University Online








4
Academics








5
Affiliated colleges








6
Campus and infrastructure








7
Research grants and memoranda of un

c:\Users\yugpa\OneDrive\Documents\AI_Summarization_project\Environment\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\yugpa\OneDrive\Documents\AI_Summarization_project\Environment\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yugpa\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run P



> Entering new LLMChain chain...
Prompt after formatting:
Combine the following summaries into a coherent overall summary:

Parul University is a private university in Vadodara, Gujarat, India. 

**Key points:**

* **Established:** 2009 (as a university), but its roots go back to 1993 with the founding of the Ahmedabad Homeopathic College.
* **Location:** 150-acre campus in Vadodara.
* **Student population:** Over 50,000 students.
* **History:**
    * Gujarat's first self-financed institute.
    * Expanded from its original location in Ahmedabad to other cities like Rajkot before settling in Vadodara in 2003.
* **Structure:**
    * Governed by the Parul Arogya Seva Mandal Trust.
    * Led by President Devanshu Patel and Vice-President Parul Patel.


The Wikipedia article highlights the university's growth and achievements while also mentioning some concerns about potential bias and reliance on primary sources. 


Parul University, established in 2015, is a rapidly growing institution

{'input_documents': [Document(metadata={'source': 'https://en.wikipedia.org/wiki/Parul_University', 'title': 'Parul University - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nParul University - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload file\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\n Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n

In [8]:
summary['output_text']

'Parul University, established in 2009 and originally founded as a charitable trust in 1989, is a rapidly growing private university in Vadodara, Gujarat, India.  It offers over 450 programs across various disciplines, including science, medicine, engineering, law, management, and the arts. \n\nThe university is known for its commitment to innovation and research, ranking among the top 50 universities in India for innovation and receiving NAAC A++ accreditation. It actively promotes international collaborations, hosting international staff teaching programs and partnering with foreign universities. \n\nParul University boasts a 150-acre campus with modern infrastructure and facilities, and it is a Category 1 University with Graded Autonomy recognized by the UGC, AICTE, NBA, and other national bodies. Its hospital, Parul Sevashram Hospital, is NABH accredited.  The university places a strong emphasis on student entrepreneurship and global cultural harmony through events like the Interna